# Test Audio Sender

This file is used for sending audio data to the device. It measures the accuracy based on individual audio files.

**Note that the software on the MCU must match this!**

In [2]:
AUDIO_FOLDER_PATH = "./dataset_mix"
POSITIVE_PATH = f"{AUDIO_FOLDER_PATH}/positive"
NEGATIVE_PATH = f"{AUDIO_FOLDER_PATH}/negative"

In [3]:
import numpy as np
import pandas as pd
import gc
import os
import librosa
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import serial
import struct

2025-04-12 15:21:49.177377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744464109.200445   11010 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744464109.206927   11010 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-12 15:21:49.229494: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/fridh/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. 

In [4]:
# Collecte positive audio files and their classes
positive_audio_files = []
positive_audio_file_classes = []
# Look each file in the audio folder
audio_class_folders = os.listdir(POSITIVE_PATH)
# Loop each class folder
for audio_class_folder in audio_class_folders:
    # Assemble full audio class folder path.
    audio_class_folder_path = os.path.join(POSITIVE_PATH, audio_class_folder)
    print("Processing class folder: ", audio_class_folder_path)
    # Get all files in the audio class folder
    audio_class_files = os.listdir(audio_class_folder_path)
    # Loop each audio file in the audio class folder
    for audio_class_file in audio_class_files:
        # Assemble full audio file path.
        audio_file_path = os.path.join(audio_class_folder_path, audio_class_file)
        # Append the audio file path to the positive_audio_files list
        positive_audio_files.append(audio_file_path)
        # Append the audio class to the audio_file_classes list
        positive_audio_file_classes.append(audio_class_folder)
print("positive_audio_files length: ", len(positive_audio_files))
print("positive_audio_file_classes length: ", len(positive_audio_file_classes))

Processing class folder:  ./dataset_mix/positive/Car
Processing class folder:  ./dataset_mix/positive/Comm
Processing class folder:  ./dataset_mix/positive/Motorcycle
positive_audio_files length:  18597
positive_audio_file_classes length:  18597


In [5]:
positive_audio_file_classes[0]

'Car'

In [6]:
# Collecte negative audio files and their classes
negative_audio_files = []
negative_audio_file_classes = []
# Look each file in the audio folder
audio_class_folders = os.listdir(NEGATIVE_PATH)
# Loop each class folder
for audio_class_folder in audio_class_folders:
    # Assemble full audio class folder path.
    audio_class_folder_path = os.path.join(NEGATIVE_PATH, audio_class_folder)
    print("Processing class folder: ", audio_class_folder_path)
    # Get all files in the audio class folder
    audio_class_files = os.listdir(audio_class_folder_path)
    # Loop each audio file in the audio class folder
    for audio_class_file in audio_class_files:
        # Assemble full audio file path.
        audio_file_path = os.path.join(audio_class_folder_path, audio_class_file)
        # Append the audio file path to the negative_audio_files list
        negative_audio_files.append(audio_file_path)
        # Append the audio class to the audio_file_classes list
        negative_audio_file_classes.append(audio_class_folder)
print("negative_audio_files length: ", len(negative_audio_files[0:10]))
print("negative_audio_file_classes length: ", len(negative_audio_file_classes[0:10]))

Processing class folder:  ./dataset_mix/negative/background
negative_audio_files length:  10
negative_audio_file_classes length:  10


In [7]:
# Shuffle the audio files and classes with the same seed.
seed = 42
np.random.seed(seed)
np.random.shuffle(positive_audio_files)

np.random.seed(seed)
np.random.shuffle(positive_audio_file_classes)

np.random.seed(seed)
np.random.shuffle(negative_audio_files)

In [8]:
# Hot end code the labels.
label_encoder = LabelEncoder()
positive_audio_file_classes_categorical = to_categorical(label_encoder.fit_transform(positive_audio_file_classes))
print("Example of audio_file_classes_categorial: ", positive_audio_file_classes_categorical[0:10])

Example of audio_file_classes_categorial:  [[0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [9]:
# Connect to MCU via serial.
print("Configuring serial port...")
ser = serial.Serial(
    port='/dev/ttyACM0',  # Change this to your actual port, e.g., 'COM3' on Windows, '/dev/ttyS0' on Linux
    baudrate=921600,       # Set baud rate to 921600
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=0.1,           # Set timeout for reading
    write_timeout=None     # Wait indefinitely until all data is sent
)

Configuring serial port...


In [10]:
if ser.is_open:
    print(f"Serial port {ser.port} opened at {ser.baudrate} baud.")

Serial port /dev/ttyACM0 opened at 921600 baud.


In [11]:
def calc_accuracy(y_true, y_pred):
    success = 0
    for i in range(len(y_pred)):
        if y_pred[i] == y_true[i]:
            success += 1
    res = round(success / len(y_pred), 3)
    return res

In [12]:
audioSent = 0
audioFilesProcessed = 0
results = []
realResults = []

In [13]:
positive_audio_file_classes_categorical[0]

array([0., 1., 0.])

In [14]:
def streamAudioFile(file_path, realClass, i):
    global audioSent
    global results
    global realResults
    global audioBuffer
    voted = False
    # Append the real result to the results list.
    realResults.append(np.argmax(realClass))
    #print(f"Adding realResults: {realResults}")
    # Check file format.
    if not file_path.endswith('.wav'):
        print("Error: Only .wav files are supported.")
        return
    # Load the audio file.
    audio_data, sr = librosa.load(file_path, sr=None)
    # Convert sample rate to 16kHz.
    if sr != 16000:
        audio_data = librosa.resample(y=audio_data, orig_sr=sr, target_sr=16000)
        sr = 16000
    # Convert the audio data to a numpy array.
    audio_data = np.array(audio_data, dtype=np.float32)
    # Send the audio data to the MCU.
    for j in range(len(audio_data)):
        data_to_send = audio_data[j].tobytes()
        ser.write(data_to_send)

        while ser.in_waiting > 0:
            response = ser.readline().decode('utf-8').strip()
            if response.startswith("e:"):
                print(response)
                return
            if response.startswith("c:"):
                print(response)
            if response.startswith("s:"):
                print(response)
            if response.startswith("v:"):
                voted = True
                print(response)
                #print(f"Adding {int(response[2:])} to results")
                response = int(response[2:])
                results.append(response)
                if len(results) > len(realResults):
                    realResults.append(9)
                print(f"Accuracy: {calc_accuracy(realResults, results)}")
    # If result is shorter than realResults, append 9 to results.
    if i > 0 and len(results) < len(realResults) and not voted:
        results.append(9)
        print(f"Accuracy: {calc_accuracy(realResults, results)}")

In [15]:

total_positive_audio_files = len(positive_audio_files)

# Loop each audio file.
negative_audio_file_pointer = 0
for i in range(total_positive_audio_files):
    # Get audio file path.
    audio_file = positive_audio_files[i]
    # Print audio file path and class.
    print("Processing audio file: ", audio_file)
    print("Processing audio file class: ", positive_audio_file_classes[i])
    # Stream positive audio file to MCU.
    streamAudioFile(audio_file, positive_audio_file_classes_categorical[i], i)
    # Print nagtive audio file path and class.
    print("Processing negative audio file: ", negative_audio_files[negative_audio_file_pointer])
    # Stream negative audio file to MCU.
    streamAudioFile(negative_audio_files[negative_audio_file_pointer], [0,0,0,1], i)
    # Increment the negative audio file pointer.
    negative_audio_file_pointer = (negative_audio_file_pointer + 1) % len(negative_audio_files)

Processing audio file:  ./dataset_mix/positive/Comm/Y99CehZOeqlQ_30.000_40.000.wav
Processing audio file class:  Comm
c: [-41,-112,9,-112] voted for: 1
c: [-152,-225,91,-226] voted for: 3
c: [-248,-337,156,-339] voted for: 3
c: [-268,-449,144,-450] voted for: 1
c: [-373,-563,224,-566] voted for: 3
c: [-452,-676,275,-681] voted for: 3
c: [-566,-791,363,-797] voted for: 3
v:2
Accuracy: 0.0
c: [-472,-670,292,-685] voted for: 3
c: [-388,-782,179,-800] voted for: 3
c: [-339,-872,79,-914] voted for: 3
c: [-435,-986,148,-1030] voted for: 3
c: [-347,-1101,34,-1145] voted for: 1
c: [-436,-1215,95,-1260] voted for: 3
v:2
Accuracy: 0.0
c: [88,-653,-283,-689] voted for: 3
c: [142,-763,-369,-804] voted for: 3
c: [31,-878,-282,-920] voted for: 3
c: [-77,-993,-200,-1036] voted for: 3
c: [-190,-1106,-116,-1150] voted for: 3
c: [-220,-1204,-131,-1263] voted for: 3
c: [-264,-1269,-180,-1361] voted for: 3
c: [-354,-1371,-130,-1476] voted for: 3
c: [-463,-1482,-53,-1590] voted for: 3
c: [-406,-1593,-140,-

KeyboardInterrupt: 

In [ ]:
# Close the serial port
ser.close()